# **Defining Model Architecture**

In [0]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

class landmark_detection:
  def __init__(self):
    pass
  
  def getModel(self):
    model=Sequential()
    model.add(Convolution2D(32,(5,5), input_shape=(96,96,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(30))
    
    return model
  
  def modelCompile(self, model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
  def modelTrain(self, model, X_train, Y_train, epochs=100):
    return model.fit(X_train, Y_train, epochs=epochs, batch_size=200, verbose=1, validation_split=0.2)
  
  def modelSave(self, model, filename):
    model.save(filename+'.h5')
    
  def loadModel(self, filename):
    return load_model(filename+'.h5')

# **Training the model to detect landmarks**

In [0]:
import pandas as pd
import time
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.models import load_model
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

def datal(df):
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))
    df = df.dropna()
    X = np.vstack(df['Image'].values) / 255.
    X = X.astype(np.float32)
    X = X.reshape(-1, 96, 96, 1)
    y = df[df.columns[:-1]].values
    y = (y - 48) / 48
    X, y = shuffle(X, y, random_state=42)
    y = y.astype(np.float32)
    return X, y


start_time = time.time()
train=pd.read_csv("training.csv")
x_train, y_train= datal(train)

obj=landmark_detection()
model=obj.getModel()
obj.modelCompile(model,'adam', 'mean_squared_error', ['accuracy'])
hist=obj.modelTrain(model, x_train, y_train)
obj.modelSave(model,"landmarkmodel")

end_time = time.time()
print(end_time - start_time)

# **Adding filter to the video and creating the new video**

In [0]:
import cv2
import numpy as np

def convert_frames_to_video(fps,frame_myself):
    pathOut = 'video_no_demotpetcetc.avi'
    frame_array = []
    size=(0,0)
    for i in frame_myself:
        height, width, layers = i.shape
        size = (width,height)
        frame_array.append(i)
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for i in range(len(frame_array)):
         out.write(frame_array[i])
    out.release()

my_model = landmark_detection().loadModel('landmarkmodel')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

filters = ['sunglasses.png', 'sunglasses_2.png', 'sunglasses_3.jpg', 'sunglasses_4.png', 'sunglasses_5.jpg', 'sunglasses_6.png']
filterIndex = 2
videofilename="landmark detection.mp4"
cap = cv2.VideoCapture(videofilename)
frame_myself=[]
while(cap.isOpened()):
    (grabbed, frame)= cap.read()
    if frame is None:
      break
    frame = cv2.flip(frame, 1)
    frame2 = np.copy(frame)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.25, 6)
    for (x, y, w, h) in faces:
        gray_face = gray[y:y+h, x:x+w]
        color_face = frame[y:y+h, x:x+w]
        gray_normalized = gray_face / 255
        original_shape = gray_face.shape
        face_resized = cv2.resize(gray_normalized, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_copy = face_resized.copy()
        face_resized = face_resized.reshape(1, 96, 96, 1)
        keypoints = my_model.predict(face_resized)
        keypoints = keypoints * 48 + 48
        face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
        face_resized_color2 = np.copy(face_resized_color)
        points = []
        for i, co in enumerate(keypoints[0][0::2]):
            points.append((co, keypoints[0][1::2][i]))
        sunglasses = cv2.imread(filters[filterIndex], cv2.IMREAD_UNCHANGED)
        sunglass_width = int((points[7][0]-points[9][0])*1.1)
        sunglass_height = int((points[10][1]-points[8][1])/1.1)
        sunglass_resized = cv2.resize(sunglasses, (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)
        transparent_region = sunglass_resized[:,:,:3] != 0
        face_resized_color[int(points[9][1]):int(points[9][1])+sunglass_height, int(points[9][0]):int(points[9][0])+sunglass_width,:][transparent_region] = sunglass_resized[:,:,:3][transparent_region]
        frame[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)
        for keypoint in points:
            cv2.circle(face_resized_color2, keypoint, 1, (0,255,0), 1)
        frame2[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, interpolation = cv2.INTER_CUBIC)
        frame_myself.append(frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
fps = 20.0
convert_frames_to_video(fps,frame_myself)